In [2]:
import pandas as pd
import pickle
from datetime import datetime
import numpy as np
import re

class CustomerDataExtractor:
    def __init__(self, orders_file_path, vip_file_path):
        self.orders_file_path = orders_file_path
        self.vip_file_path = vip_file_path
        self.category_mapping = {1: 'Electronics', 2: 'Apparel', 3: 'Books', 4: 'Home Goods'}
    def load_vip_customers(self):
        vip_customers = set()
        with open(self.vip_file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:
                    vip_customers.add(int(line))
        return vip_customers
    def load_customer_orders(self):
        with open(self.orders_file_path, 'rb') as file:
            customer_records = pickle.load(file)
        return customer_records
    def convert_price_to_float(self, price_value):
        if isinstance(price_value, (int, float)):
            return float(price_value)
        elif isinstance(price_value, str):
            #check if we can make it a float by removing non-numeric characters
            cleaned_price = re.sub(r'[^\d.]', '', price_value)
            try:
                return float(cleaned_price)
            except ValueError:
                #if not return 0.0
                return 0.0
        else:
            return 0.0
    def convert_category(self, category_value):
        try:
            category_int = int(category_value)
            return self.category_mapping.get(category_int, 'Misc')
        except (ValueError, TypeError):
            return 'Misc'
    def extract_flat_data(self):
        vip_customers = self.load_vip_customers()
        customer_records = self.load_customer_orders()
        flat_data = []
        for customer in customer_records:
            customer_id = customer['id']
            customer_name = customer['name']
            registration_date = pd.to_datetime(customer['registration_date'], errors='coerce')
            is_vip = customer_id in vip_customers
            orders = customer.get('orders', [])
            #if orders is not a list or is empty, skip it
            if not isinstance(orders, list) or not orders:
                continue
            for order in orders:
                order_id = order['order_id']
                if order_id is None:
                    continue
                order_date = pd.to_datetime(order['order_date'], errors='coerce')
                items = order.get('items', [])
                #if items is not a list or is empty, skip it
                if not isinstance(items, list) or not items:
                    continue
                order_total_value = order.get('order_total_value', None)
                if order_total_value is not None and order_total_value > 0:
                    total_order_value = float(order_total_value)
                else:
                    #falback to 0.0
                    total_order_value = 0.0
                    for item in items:
                        price = self.convert_price_to_float(item['price'])
                        quantity = int(item['quantity']) if isinstance(item['quantity'], (int, float)) else 0 #again fallback to 0
                        total_order_value += price * quantity
                for item in items:
                    item_id = item['item_id']
                    product_name = item['product_name']
                    category = self.convert_category(item['category'])
                    unit_price = self.convert_price_to_float(item['price'])
                    item_quantity = int(item['quantity']) if isinstance(item['quantity'], (int, float)) else 0 #same here
                    total_item_price = unit_price * item_quantity
                    total_order_value_percentage = (total_item_price / total_order_value * 100) if total_order_value > 0 else 0.0 #same here
                    flat_data.append({
                        'customer_id': customer_id,
                        'customer_name': customer_name,
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': order_id,
                        'order_date': order_date,
                        'product_id': item_id,
                        'product_name': product_name,
                        'category': category,
                        'unit_price': unit_price,
                        'item_quantity': item_quantity,
                        'total_item_price': total_item_price,
                        'total_order_value_percentage': total_order_value_percentage
                    })
        df = pd.DataFrame(flat_data)
        df['customer_id'] = pd.to_numeric(df['customer_id'], errors='coerce')
        df['order_id'] = pd.to_numeric(df['order_id'], errors='coerce')
        df['product_id'] = pd.to_numeric(df['product_id'], errors='coerce')
        df = df.dropna(subset=['customer_id', 'order_id', 'product_id']) #drop rows with missing values
        df['customer_id'] = df['customer_id'].astype(int)
        df['order_id'] = df['order_id'].astype(int)
        df['product_id'] = df['product_id'].astype(int)
        df['customer_name'] = df['customer_name'].astype(str)
        df['registration_date'] = pd.to_datetime(df['registration_date'])
        df['is_vip'] = df['is_vip'].astype(bool)
        df['order_date'] = pd.to_datetime(df['order_date'])
        df['product_name'] = df['product_name'].astype(str)
        df['category'] = df['category'].astype(str)
        df['unit_price'] = df['unit_price'].astype(float)
        df['item_quantity'] = df['item_quantity'].astype(int)
        df['total_item_price'] = df['total_item_price'].astype(float)
        df['total_order_value_percentage'] = df['total_order_value_percentage'].astype(float)
        df = df.sort_values(['customer_id', 'order_id', 'product_id'], ascending=[True, True, True]) #sort by ascending, ascending is true by default but it's good to be explicit
        return df

def main():
    extractor = CustomerDataExtractor('customer_orders.pkl', 'vip_customers.txt')
    print("Extracting customer order data")
    df = extractor.extract_flat_data()
    print(f"\nExtraction completed successfully")
    print(f"\nFirst 5 rows of the extracted data:")
    print(df.head())
    output_file = 'customer_orders_extracted.csv'
    df.to_csv(output_file, index=False)
    print(f"\nData saved to {output_file}")
    return df

if __name__ == "__main__":
    df = main()

Extracting customer order data

Extraction completed successfully

First 5 rows of the extracted data:
   customer_id customer_name   registration_date  is_vip  order_id  \
0            1    Customer 1 2022-12-31 04:19:19    True         3   
1            2    Customer 2 2022-05-27 00:23:28   False         5   
2            2    Customer 2 2022-05-27 00:23:28   False         5   
3            2    Customer 2 2022-05-27 00:23:28   False         5   
4            2    Customer 2 2022-05-27 00:23:28   False         5   

           order_date  product_id        product_name     category  \
0 2023-09-06 00:42:50           1  Item 1 for Order 3  Electronics   
1 2024-08-28 19:37:56           1  Item 1 for Order 5         Misc   
2 2024-08-28 19:37:56           2  Item 2 for Order 5      Apparel   
3 2024-08-28 19:37:56           3  Item 3 for Order 5  Electronics   
4 2024-08-28 19:37:56           4  Item 4 for Order 5         Misc   

   unit_price  item_quantity  total_item_price  total_o